<a href="https://colab.research.google.com/github/DivyanshiSingh12/Meeting_Info_Bot/blob/master/Streamlit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 30.2 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import re
import nltk
nltk.download('punkt')
import streamlit as st
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


def generate_summary(conversation):
    # Extract participant names and their statements from the conversation
    participants_statements = re.findall(r'(Person [A-Z]): "(.*?)"', conversation)

    # Generate summary of what each participant has said
    summary = {}
    for participant, statement in participants_statements:
        if participant not in summary:
            summary[participant] = []
        summary[participant].append(statement)

    # Generate the summary text
    summary_text = ""
    for participant, statements in summary.items():
        summary_text += f"{participant}:\n"
        summary_sentences = []
        for statement in statements:
            sentences = sent_tokenize(statement)
            summary_sentences.extend(sentences)
        summary_text += " ".join(summary_sentences)
        summary_text += "\n\n"

    return summary_text


def preprocess_text(text):
    # Remove special characters and extra whitespaces
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = re.sub(r"\s+", " ", text)
    # Convert to lowercase
    text = text.lower()
    return text

examples = [
("Hi everyone! Thank you all for being here today. Let's discuss the needs and requirements for our new startup.", "Discussing the need for a solid business plan"),
("Hi everyone! Thank you all for being here today. Let's discuss the needs and requirements for our new startup.", "Discussing the need for a solid business plan"),
("Absolutely, a well-defined business plan is crucial. We also need to consider our financial requirements. How much funding will we need to get started and sustain the business until we become profitable?", "Determining financial requirements and funding needs"),
("Agreed. We should conduct a thorough market analysis to determine the potential demand for our product or service. This will help us gauge the size of our target market and estimate the sales and revenue projections.", "Conducting market analysis and estimating sales projections"),
("Additionally, we need to identify the right technology infrastructure and tools to support our operations. We should discuss the hardware, software, and network requirements for our startup.", "Identifying technology infrastructure and tools required"),
("I think branding and marketing will play a significant role in our success. We need to establish a strong brand identity and develop a comprehensive marketing strategy to reach our target audience effectively.", "Establishing a strong brand identity and marketing strategy"),
("Good point! We should also consider the talent and skills required to run the startup. Let's discuss the roles and responsibilities we need to fill and create a hiring plan.", "Planning roles, responsibilities, and hiring needs"),
("Speaking of talent, we should invest in ongoing training and development programs to ensure our team stays up-to-date with the latest industry trends and skills.", "Investing in ongoing training and development programs"),
("Absolutely, and we shouldn't forget about legal and regulatory compliance. We need to understand the laws and regulations that apply to our business and ensure we have the necessary licenses and permits.", "Ensuring legal and regulatory compliance"),
("Another important aspect is scalability. As we grow, our startup should be able to handle increased demand. We need to plan for scalability in terms of infrastructure, staffing, and processes.", "Planning for scalability as the startup grows"),
("And let's not overlook the importance of customer support. We should prioritize building strong customer relationships and providing exceptional support to ensure customer satisfaction.", "Prioritizing customer support and satisfaction"),
("Good point! Let's create an action plan to tackle each of these requirements and set realistic timelines to achieve them.", "Creating an action plan and setting realistic timelines"),
("Agreed. We should assign responsibilities to each team member to ensure accountability and progress. Regular check-ins and milestones will help us stay on track.", "Assigning responsibilities and ensuring accountability"),
("Absolutely. By addressing these needs and requirements systematically, we'll be on our way to building a successful and sustainable startup.", "Evaluating progress and adapting plans as needed"),
("I'm excited about this journey with all of you. Let's bring our expertise together and make this startup a remarkable success!", "Building a successful and sustainable startup"),
("Let's analyze our target market and identify the key demographics and psychographics we should focus on.", "Conducting market analysis and estimating sales projections"),
("We need to invest in robust cybersecurity measures to protect our data and customer information from potential threats.", "Identifying technology infrastructure and tools required"),
("To establish our brand identity, we should create a compelling logo, design captivating visuals, and craft a unique brand voice.", "Establishing a strong brand identity and marketing strategy"),
("We should outline the core competencies required for each role and define the qualifications and experience we seek in potential candidates.", "Planning roles, responsibilities, and hiring needs"),
("Let's explore partnerships with educational institutions and industry experts to provide specialized training programs for our team.", "Investing in ongoing training and development programs"),
("We should consult with legal experts to ensure compliance with data privacy laws, intellectual property regulations, and any industry-specific regulations.", "Ensuring legal and regulatory compliance"),
("As part of our scalability plan, we should consider cloud-based solutions that can easily accommodate increased user traffic and data storage needs.", "Planning for scalability as the startup grows"),
("We need to develop a customer support strategy that includes multiple channels for communication and a system for tracking and resolving customer issues.", "Prioritizing customer support and satisfaction"),
("Let's assign specific tasks to each team member and establish a project management system to track progress and deadlines.", "Creating an action plan and setting realistic timelines"),
("Regular performance evaluations and feedback sessions will help us identify areas for improvement and recognize outstanding contributions.", "Evaluating progress and adapting plans as needed"),
("Building a strong company culture that fosters collaboration, innovation, and a sense of belonging will be crucial for our startup's long-term success.", "Building a successful and sustainable startup"),
("Idea and Product Development: Start with a strong and innovative idea for your product or service.", "Idea and Product Development"),
("Conduct market research to validate the demand and viability of your idea.", "Idea and Product Development"),
("Develop a minimum viable product (MVP) or prototype to test and gather feedback from potential customers.", "Idea and Product Development"),
("Continuously iterate and refine your product based on user feedback and market dynamics.", "Idea and Product Development"),
("Brainstorm creative concepts and innovative solutions for your product or service.", "Idea and Product Development"),
("Analyze customer pain points and identify opportunities for addressing them through your product.", "Idea and Product Development"),
("Conduct user interviews and surveys to gain insights into customer needs and preferences.", "Idea and Product Development"),
("Collaborate with a multidisciplinary team to ideate and generate diverse ideas for product development.", "Idea and Product Development"),
("Use design thinking methodologies to empathize with users and generate user-centric product ideas.", "Idea and Product Development"),
("Evaluate the feasibility and technical viability of your product ideas before proceeding with development.", "Idea and Product Development"),
("Create a detailed roadmap outlining the development stages and milestones for your product.", "Idea and Product Development"),
("Implement agile development methodologies to ensure rapid iteration and responsiveness to market changes.", "Idea and Product Development"),
("Prototype different versions of your product to gather user feedback and validate key features.", "Idea and Product Development"),
("Leverage emerging technologies and trends to enhance the uniqueness and competitiveness of your product.", "Idea and Product Development"),
("Identify your target market segments and understand their demographics and preferences.", "Market Analysis and Strategy"),
("Analyze market trends and competitive landscape to identify market opportunities and challenges.", "Market Analysis and Strategy"),
("Conduct thorough market research to assess the size and growth potential of your target market.", "Market Analysis and Strategy"),
("Develop a clear value proposition that differentiates your product from competitors in the market.", "Market Analysis and Strategy"),
("Define your target customer personas to better understand their needs, motivations, and pain points.", "Market Analysis and Strategy"),
("Identify effective marketing channels and strategies to reach your target audience.", "Market Analysis and Strategy"),
("Perform a SWOT analysis to evaluate your startup's strengths, weaknesses, opportunities, and threats.", "Market Analysis and Strategy"),
("Determine pricing strategies that align with market dynamics and customer perceptions.", "Market Analysis and Strategy"),
("Create a comprehensive marketing plan that outlines your promotional activities and messaging.", "Market Analysis and Strategy"),
("Assess the regulatory and legal requirements relevant to your product or industry.", "Market Analysis and Strategy"),
("Monitor and analyze key performance indicators (KPIs) to track your market performance.", "Market Analysis and Strategy"),
("Identify strategic partnerships or collaborations that can help expand your market reach.", "Market Analysis and Strategy"),
("Evaluate potential risks and contingencies to develop a robust risk management plan.", "Market Analysis and Strategy"),
("Segment your market based on customer needs and preferences to customize your offerings.", "Market Analysis and Strategy"),
("Conduct competitor analysis to understand their strengths, weaknesses, and market positioning.", "Market Analysis and Strategy"),
("Stay updated with emerging market trends and consumer behavior to stay ahead of the competition.", "Market Analysis and Strategy"),
("Adapt your marketing and sales strategies based on the feedback and insights gathered from the market.", "Market Analysis and Strategy"),
("Develop a strong brand identity and positioning that resonates with your target market.", "Market Analysis and Strategy"),
("Identify potential distribution channels and partners to effectively reach your target customers.", "Market Analysis and Strategy"),
("Continuously refine your market strategy based on market feedback and evolving customer needs.", "Market Analysis and Strategy"),
("Determine your startup's financial needs and create a detailed budget plan.", "Funding and Financial Management"),
("Explore various funding options such as angel investors, venture capital, or crowdfunding.", "Funding and Financial Management"),
("Prepare a compelling business plan and financial projections to attract potential investors.", "Funding and Financial Management"),
("Conduct thorough due diligence when evaluating potential investors or funding sources.", "Funding and Financial Management"),
("Establish financial controls and implement accounting systems to track income and expenses.", "Funding and Financial Management"),
("Monitor and manage your cash flow effectively to ensure smooth operations.", "Funding and Financial Management"),
("Consider bootstrapping as an initial funding strategy to minimize reliance on external capital.", "Funding and Financial Management"),
("Develop financial models to analyze different scenarios and assess the profitability of your startup.", "Funding and Financial Management"),
("Create a comprehensive fundraising strategy that outlines your target funding milestones and timelines.", "Funding and Financial Management"),
("Negotiate favorable terms with investors while considering the long-term impact on your startup's growth.", "Funding and Financial Management"),
("Implement cost control measures to optimize expenses and increase your startup's profitability.", "Funding and Financial Management"),
("Consider alternative funding options such as grants, business loans, or strategic partnerships.", "Funding and Financial Management"),
("Establish relationships with financial advisors or mentors who can provide guidance on funding and financial management.", "Funding and Financial Management"),
("Monitor key financial metrics such as burn rate, runway, and customer acquisition cost.", "Funding and Financial Management"),
("Prepare and present financial reports to stakeholders and investors to demonstrate transparency and accountability.", "Funding and Financial Management"),
("Develop contingency plans and financial risk mitigation strategies to address potential challenges.", "Funding and Financial Management"),
("Optimize your capital structure by balancing equity and debt financing options.", "Funding and Financial Management"),
("Consider the tax implications and legal requirements associated with fundraising and financial management.", "Funding and Financial Management"),
("Periodically review and revise your financial plan to align with the changing needs and goals of your startup.", "Funding and Financial Management"),
("Engage in investor relations activities to maintain strong relationships with existing investors and attract new ones.", "Funding and Financial Management"),
("Define the roles and responsibilities within your startup to establish a clear organizational structure.", "Team Building and Talent Acquisition"),
("Identify the key skills and expertise required for each role in your startup.", "Team Building and Talent Acquisition"),
("Create a compelling company culture and values that attract and retain top talent.", "Team Building and Talent Acquisition"),
("Develop a robust recruitment strategy to attract qualified candidates.", "Team Building and Talent Acquisition"),
("Leverage various channels, such as job boards and professional networks, to source potential candidates.", "Team Building and Talent Acquisition"),
("Implement a rigorous interview and selection process to ensure the right fit for your team.", "Team Building and Talent Acquisition"),
("Provide competitive compensation and benefits packages to attract and retain top talent.", "Team Building and Talent Acquisition"),
("Invest in employee development programs and opportunities for growth.", "Team Building and Talent Acquisition"),
("Promote diversity and inclusion within your startup to foster a vibrant and innovative team.", "Team Building and Talent Acquisition"),
("Build strong relationships with universities and industry organizations to tap into emerging talent pools.", "Team Building and Talent Acquisition"),
("Implement effective onboarding processes to help new hires integrate into the team smoothly.", "Team Building and Talent Acquisition"),
("Nurture a collaborative work environment that encourages teamwork and knowledge sharing.", "Team Building and Talent Acquisition"),
("Provide regular feedback and recognition to motivate and engage your team members.", "Team Building and Talent Acquisition"),
("Develop leadership development programs to identify and nurture future leaders within your startup.", "Team Building and Talent Acquisition"),
("Promote a healthy work-life balance and employee well-being to create a positive work environment.", "Team Building and Talent Acquisition"),
("Create mentorship programs to facilitate knowledge transfer and professional growth.", "Team Building and Talent Acquisition"),
("Implement performance management systems to set clear goals and track individual and team performance.", "Team Building and Talent Acquisition"),
("Encourage ongoing learning and skill development through training programs and workshops.", "Team Building and Talent Acquisition"),
("Cultivate a strong employer brand to attract top talent and differentiate your startup from competitors.", "Team Building and Talent Acquisition"),
("Regularly assess and evaluate the team dynamics to identify areas for improvement and foster a cohesive and high-performing team.", "Team Building and Talent Acquisition"),
("Translate your startup vision into actionable goals and develop a detailed execution plan.", "Execution and Adaptability"),
("Assign clear responsibilities and timelines to team members to ensure efficient execution of tasks.", "Execution and Adaptability"),
("Implement agile project management methodologies to enable flexibility and adaptability to market changes.", "Execution and Adaptability"),
("Monitor and track key performance indicators (KPIs) to measure progress and make data-driven decisions.", "Execution and Adaptability"),
("Establish regular communication channels to facilitate seamless collaboration and information sharing.", "Execution and Adaptability"),
("Create a culture of accountability and ownership to drive execution and meet deadlines.", "Execution and Adaptability"),
("Anticipate potential obstacles and develop contingency plans to navigate challenges effectively.", "Execution and Adaptability"),
("Regularly review and assess your startup's progress and adjust strategies accordingly.", "Execution and Adaptability"),
("Encourage a mindset of experimentation and learning from failures to drive continuous improvement.", "Execution and Adaptability"),
("Stay updated with market trends and customer feedback to adapt your product or service offerings.", "Execution and Adaptability"),
("Embrace change and pivot when necessary to align with evolving market demands and opportunities.", "Execution and Adaptability"),
("Encourage cross-functional collaboration and knowledge sharing to foster adaptability and innovation.", "Execution and Adaptability"),
("Continuously seek feedback from customers and iterate on your product or service based on their needs.", "Execution and Adaptability"),
("Monitor competitor activities and industry trends to identify potential areas for improvement and adaptation.", "Execution and Adaptability"),
("Promote a culture of continuous learning and professional development within your startup.", "Execution and Adaptability"),
("Regularly assess and optimize internal processes to streamline operations and enhance efficiency.", "Execution and Adaptability"),
("Encourage a growth mindset that embraces change, challenges, and continuous improvement.", "Execution and Adaptability"),
("Build a diverse and adaptable team that can readily respond to changing market dynamics.", "Execution and Adaptability"),
("Leverage technology and automation tools to streamline workflows and increase agility.", "Execution and Adaptability"),
("Perform thorough market research to identify customer pain points and develop a product that addresses their needs.", "Idea and Product Development"),
("Stay agile and responsive to customer needs by incorporating feedback into your product development cycle.", "Execution and Adaptability"),
("Analyze the competitive landscape to identify opportunities and differentiate our product in the market.", "Market Analysis and Strategy"),
("Secure funding through a combination of venture capital, angel investors, and crowdfunding.", "Funding and Financial Management"),
("Implement a performance-based compensation system to attract top talent and motivate the team.", "Team Building and Talent Acquisition"),
("Monitor key financial metrics and maintain accurate financial records for effective financial management.", "Funding and Financial Management"),
("Create a strong brand identity and communicate our unique value proposition to our target audience.", "Market Analysis and Strategy"),
("Build a diverse and skilled team by actively recruiting talent from various sources.", "Team Building and Talent Acquisition"),
("Adapt our marketing strategy based on market trends and customer feedback to maximize our reach.", "Execution and Adaptability"),
("Regularly evaluate and optimize our pricing strategy to remain competitive in the market.", "Market Analysis and Strategy"),
("Stay agile and responsive to customer needs by incorporating feedback into your product development cycle.", "Execution and Adaptability")

]

# Split the examples into inputs (text) and labels (intents)
inputs = [example[0] for example in examples]
labels = [example[1] for example in examples]

# Preprocess the inputs (you can customize this based on your requirements)
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(inputs)

# Split the dataset into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train the intent recognition model (SVM classifier)
classifier = SVC(kernel='linear')
classifier.fit(features_train, labels_train)


def main():
    # Set title and description for the Streamlit app
    st.title("Meeting Summary Generator")
    record_button = st.button("Record")
    stop_button = st.button("Stop")

    st.markdown("Enter the conversation below and click the 'Generate Summary' button to generate the summary of what each participant has said.")

    # Create a text area for the user to enter the conversation
    conversation = st.text_area("Conversation", value="", height=400)



    generate_summary_button = st.button("Summary")
    determine_intent_button = st.button("Intent")



    if record_button:
        # Start recording logic goes here
        st.write("Recording started...")

    if stop_button:
        # Stop recording logic goes here
        st.write("Recording stopped.")

    if generate_summary_button:
        # Generate the summary text
        summary_text = generate_summary(conversation)

        # Display the summary
        st.markdown("### Summary")
        st.text(summary_text)


    if determine_intent_button:
        if conversation:
            lines = conversation.split("\n")

            for line in lines:
                preprocessed_line = preprocess_text(line)
                input_feature = vectorizer.transform([preprocessed_line])
                predicted_intent = classifier.predict(input_feature)[0]
                st.write("Line:", line)
                st.write("Predicted Intent:", predicted_intent)






if __name__ == "__main__":
    main()





Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.184.2:8501

npx: installed 22 in 7.927s
your url is: https://dry-loops-do.loca.lt
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
